In [2]:
import pandas as pd
import numpy as np

In [89]:
mst = pd.read_csv('mst_data/mst_game.csv')
df = pd.DataFrame()
for i, row in mst.iterrows():
    _df = pd.read_csv(row['csv_file'])
    _df['file_id'] = i
    df = pd.concat([df, _df])
# round当たりのcountが500以上と200以下のものを削除
df = df.groupby(['file_id', 'round']).filter(lambda x: x['round'].count() > 200 and x['round'].count() < 500)

# countを350に統一
df = df.groupby(['file_id', 'round']).tail(200)

# np.arrayに変換
grouped_df = df.groupby(['file_id', 'round'])
grouped_arrays = {name: group.drop(columns=['round', 'timestamp', 'file_id']).values for name, group in grouped_df}
grouped_arrays

{(0,
  1): array([[-0.04620812, -0.65594018, -0.2629213 , ..., -0.11433781,
          0.80368322,  0.15020078],
        [-0.04620732, -0.65579033, -0.26372972, ..., -0.11539207,
          0.80390501,  0.15019406],
        [-0.04648203, -0.65585202, -0.26447821, ..., -0.1200095 ,
          0.80221063,  0.15073515],
        ...,
        [-0.05278119, -0.64836282, -0.26751611, ...,  0.0422912 ,
          0.81456858,  0.13775925],
        [-0.04914724, -0.65186507, -0.2678127 , ...,  0.04159993,
          0.81609958,  0.13849479],
        [-0.03860228, -0.65677518, -0.26826757, ...,  0.04071263,
          0.81746542,  0.13875057]]),
 (0,
  3): array([[-0.05621802, -0.65702671, -0.26297116, ..., -0.0858097 ,
          0.80783325,  0.17450878],
        [-0.05661663, -0.656712  , -0.26400465, ..., -0.08707633,
          0.80779654,  0.18032968],
        [-0.05657384, -0.65637594, -0.26454046, ..., -0.08558901,
          0.80665386,  0.18061635],
        ...,
        [-0.02516834, -0.64594722,

In [91]:
list(grouped_arrays.keys())

[(0, 1),
 (0, 3),
 (0, 5),
 (0, 7),
 (0, 8),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (2, 1),
 (2, 3),
 (2, 6),
 (2, 7),
 (2, 8),
 (3, 3),
 (3, 4),
 (3, 5),
 (3, 8),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4),
 (4, 5),
 (4, 6),
 (4, 7),
 (4, 8),
 (5, 2),
 (5, 3),
 (5, 4),
 (5, 5),
 (5, 6),
 (5, 8),
 (6, 1),
 (6, 2),
 (6, 3),
 (6, 4),
 (6, 5),
 (6, 6),
 (6, 8),
 (7, 2),
 (7, 3),
 (7, 4),
 (7, 5),
 (7, 6),
 (7, 7),
 (7, 8),
 (8, 2),
 (8, 3),
 (8, 4),
 (8, 6),
 (8, 7),
 (8, 8),
 (9, 1),
 (9, 2),
 (9, 3),
 (9, 4),
 (9, 5),
 (9, 6),
 (9, 7),
 (9, 8),
 (10, 1),
 (10, 2),
 (10, 3),
 (10, 4),
 (10, 5),
 (10, 6),
 (10, 7),
 (10, 8),
 (11, 1),
 (11, 2),
 (11, 3),
 (11, 5),
 (11, 6),
 (11, 8),
 (12, 1),
 (12, 2),
 (12, 3),
 (12, 4),
 (12, 6),
 (14, 1),
 (14, 2),
 (14, 3),
 (14, 4),
 (14, 5),
 (14, 6),
 (14, 8),
 (15, 1),
 (15, 2),
 (15, 3),
 (15, 4),
 (15, 5),
 (15, 6),
 (15, 7),
 (15, 8),
 (16, 1),
 (16, 2),
 (16, 4),
 (16, 5),
 (16, 6),
 (16, 7),
 (16, 8)]

In [98]:
X = np.array(list(grouped_arrays.values()))
Y = np.array([mst.loc[file]['player'] for file, round in list(grouped_arrays.keys())])
X.shape, Y.shape

((105, 200, 99), (105,))

In [107]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

# ダミーデータの生成
n_samples = 105
time_steps = 200
n_features = 99

# Tensorに変換
X_tensor = torch.tensor(X).to(torch.float32)
y_tensor = torch.tensor(Y).to(torch.float32)
dataset = TensorDataset(X_tensor, y_tensor)

# データセットの分割
train_size = int(0.7 * n_samples)
val_size = int(0.15 * n_samples)
test_size = n_samples - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# LSTMモデルの定義
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        h_lstm, _ = self.lstm(x)
        h_lstm_last = h_lstm[:, -1, :]  # 最後のタイムステップの出力を使用
        out = self.fc(h_lstm_last)
        return torch.sigmoid(out)

# ハイパーパラメータの設定
input_dim = n_features
hidden_dim = 50
output_dim = 1
num_epochs = 20

# モデル、損失関数、最適化手法の定義
model = LSTMClassifier(input_dim, hidden_dim, output_dim)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_losses = []
val_losses = []

# モデルの学習
for epoch in range(num_epochs):
    # 訓練モード
    model.train()
    train_loss = 0.0
    for X_batch, y_batch in train_loader:
        # 順伝播
        outputs = model(X_batch)
        outputs = outputs.squeeze(1)  # バッチ次元を維持しながら1次元を削除
        loss = criterion(outputs, y_batch)
        
        # 逆伝播と最適化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * X_batch.size(0)
    
    train_loss /= len(train_loader.dataset)
    train_losses.append(train_loss)
    
    # 検証モード
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            outputs = outputs.squeeze(1)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item() * X_batch.size(0)
            predicted = (outputs > 0.5).float()
            accuracy = (predicted == y_batch).float().mean()
    
    val_loss /= len(val_loader.dataset)
    val_losses.append(val_loss)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Accuracy: {accuracy:.4f}')

# モデルの評価（テスト）
model.eval()
test_loss = 0.0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        outputs = outputs.squeeze(1)
        loss = criterion(outputs, y_batch)
        test_loss += loss.item() * X_batch.size(0)

test_loss /= len(test_loader.dataset)
print(f'Test Loss: {test_loss:.4f}')


Epoch [1/20], Train Loss: 0.6635, Val Loss: 0.7147, Accuracy: 0.7143
Epoch [2/20], Train Loss: 0.6196, Val Loss: 0.7363, Accuracy: 0.7143
Epoch [3/20], Train Loss: 0.6201, Val Loss: 0.7587, Accuracy: 0.7143
Epoch [4/20], Train Loss: 0.6038, Val Loss: 0.7010, Accuracy: 0.7143
Epoch [5/20], Train Loss: 0.5954, Val Loss: 0.6747, Accuracy: 0.7143
Epoch [6/20], Train Loss: 0.5766, Val Loss: 0.6632, Accuracy: 0.7143
Epoch [7/20], Train Loss: 0.4595, Val Loss: 0.4847, Accuracy: 0.8571
Epoch [8/20], Train Loss: 0.2542, Val Loss: 1.3317, Accuracy: 0.7143
Epoch [9/20], Train Loss: 0.7482, Val Loss: 0.6555, Accuracy: 0.7143
Epoch [10/20], Train Loss: 0.6283, Val Loss: 0.6548, Accuracy: 0.7143
Epoch [11/20], Train Loss: 0.5976, Val Loss: 0.6988, Accuracy: 0.7143
Epoch [12/20], Train Loss: 0.5733, Val Loss: 0.6817, Accuracy: 0.7143
Epoch [13/20], Train Loss: 0.5584, Val Loss: 0.6843, Accuracy: 0.7143
Epoch [14/20], Train Loss: 0.5350, Val Loss: 0.6436, Accuracy: 0.7143
Epoch [15/20], Train Loss: 0.

In [111]:
import plotly.graph_objects as go

# 学習曲線のプロット
fig = go.Figure()

# トレーニング損失の追加
fig.add_trace(go.Scatter(
    x=list(range(1, num_epochs + 1)),
    y=train_losses,
    mode='lines+markers',
    name='Train Loss'
))

# 検証損失の追加
fig.add_trace(go.Scatter(
    x=list(range(1, num_epochs + 1)),
    y=val_losses,
    mode='lines+markers',
    name='Validation Loss'
))

# レイアウトの設定
fig.update_layout(
    title='Training and Validation Loss',
    xaxis_title='Epoch',
    yaxis_title='Loss',
    template='plotly_dark',
    width=600,
    height=500
)

# プロットの表示
fig.show()

In [31]:
X.shape

(34597, 99)